In [1]:



import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Generate synthetic data
def generate_data(num_points=50):
    X = np.linspace(0, 10, num_points)
    true_slope, true_intercept = 2.0, 1.0
    noise = np.random.normal(0, 1, num_points)
    y = true_slope * X + true_intercept + noise
    return X, y

X_data, y_data = generate_data()
slope = tf.Variable(initial_value=1.0, dtype=tf.float32, name="slope", trainable=True)
intercept = tf.Variable(initial_value=0.0, dtype=tf.float32, name="intercept", trainable=True)
# Bayesian Linear Regression Model
@tf.function
def model(X, y):
    # Define the linear regression model
    y_mean = slope * X + intercept
    
    # Specify a normal prior for the parameters
    prior = tfp.distributions.Normal(loc=0.0, scale=1.0)
    
    # Specify a likelihood function
    likelihood = tfp.distributions.Normal(loc=y_mean, scale=1.0)
    
    # Compute the log likelihood
    log_likelihood = tf.reduce_sum(likelihood.log_prob(y))
    
    # Compute the log prior
    log_prior = tf.reduce_sum(prior.log_prob(slope)) + tf.reduce_sum(prior.log_prob(intercept))
    
    # Compute the log posterior (up to a constant)
    log_posterior = log_likelihood + log_prior
    
    return log_posterior

# Bayesian Inference using Markov Chain Monte Carlo (MCMC)
num_samples = 1000
num_burnin_steps = 500

@tf.function
def run_chain(initial_state, num_results=num_samples, num_burnin_steps=num_burnin_steps):
    # Define the Markov Chain Monte Carlo (MCMC) kernel
    hmc_kernel = tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=model,
        step_size=0.1,
        num_leapfrog_steps=3
    )
    
    # Run the chain
    states, _ = tfp.mcmc.sample_chain(
        num_results=num_results,
        num_burnin_steps=num_burnin_steps,
        current_state=initial_state,
        kernel=hmc_kernel
    )
    
    return states

# Initial state for MCMC
initial_state = [tf.constant(1.0), tf.constant(0.0)]

# Run the MCMC chain
states = run_chain(initial_state)

# Extract samples from the chain
slope_samples, intercept_samples = states

# Plot the results using Bokeh
p1 = figure(title='Posterior Distribution of Slope', x_axis_label='Slope', y_axis_label='Density')
p2 = figure(title='Posterior Distribution of Intercept', x_axis_label='Intercept', y_axis_label='Density')

# Plot histograms of samples
p1.quad(top=np.histogram(slope_samples, bins=30, density=True)[0], bottom=0, left=np.histogram(slope_samples, bins=30, density=True)[1][:-1], right=np.histogram(slope_samples, bins=30, density=True)[1][1:], alpha=0.5, line_color="black")
p2.quad(top=np.histogram(intercept_samples, bins=30, density=True)[0], bottom=0, left=np.histogram(intercept_samples, bins=30, density=True)[1][:-1], right=np.histogram(intercept_samples, bins=30, density=True)[1][1:], alpha=0.5, line_color="black")

# Show the plots
grid = gridplot([[p1, p2]])
show(grid)


C:\Users\Vikas Singh B\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_probability\python\mcmc\sample.py:339: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.
  warnings.warn('Tracing all kernel results by default is deprecated. Set '
